In [1]:
import pdfplumber
import pandas as pd
from collections import defaultdict

def processar_pagina(page):
    text = page.extract_text()
    tables = page.extract_tables()
    
    # Extrai todas as linhas de texto para identificar recessos
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    
    # Dicionário para mapear dias com recesso
    recesso_dias = {}
    current_day = None
    
    for line in lines:
        # Detecta linhas que começam com dias da semana
        if any(line.lower().startswith(weekday) for weekday in ['segunda', 'terça', 'quarta', 'quinta', 'sexta']):
            current_day = line
            recesso_dias[current_day] = False
        elif line == 'RECESSO' and current_day:
            recesso_dias[current_day] = True

    # Processa tabelas normalmente
    header0 = tables[0]
    day_indices = [1, 4, 7, 10, 13]
    day_names = []
    
    for idx in day_indices:
        part1 = header0[0][idx] if idx < len(header0[0]) else ""
        part2 = header0[1][idx] if idx < len(header0[1]) else ""
        combined = f"{part1} {part2}".strip()
        day_names.append(combined)
    
    if len(day_names) != 5:
        return None

    colunas_finais = ['Categoria'] + day_names

    # Processa o corpo das tabelas
    body_raw = [row for table in tables[1:] for row in table]
    clean_body = [row for row in body_raw if row and any(cell and cell.strip() for cell in row)]

    def adjust_row(row):
        row = row + [None] * (7 - len(row))
        if (not row[5] or row[5].strip() == "") and row[6] and row[6].strip() != "":
            row[5] = row[6]
        return row[:6]

    adjusted_body = [adjust_row(row) for row in clean_body if len(row) >= 6]
    df = pd.DataFrame(adjusted_body, columns=colunas_finais)
    df['Categoria'] = df['Categoria'].ffill()

    # Transforma para formato por dia
    df_t = df.set_index('Categoria').T.reset_index()
    df_t = df_t.rename(columns={'index': 'Dia'})

    # Aplica recessos
    for dia, is_recesso in recesso_dias.items():
        if is_recesso:
            # Encontra a linha correspondente ao dia de recesso
            mask = df_t['Dia'].str.contains(dia.split(',')[0], case=False, na=False)
            # Substitui todas as categorias por 'RECESSO'
            for col in df_t.columns[1:]:
                df_t.loc[mask, col] = 'RECESSO'

    agrupadas = defaultdict(list)
    for col in df_t.columns[1:]:
        agrupadas[col].append(col)

    df_agrupado = pd.DataFrame()
    df_agrupado['Dia'] = df_t['Dia']

    for categoria, col_list in agrupadas.items():
        if len(col_list) == 1:
            df_agrupado[categoria] = df_t[col_list[0]]
        else:
            df_agrupado[categoria] = df_t[col_list].apply(
                lambda row: ' e '.join(dict.fromkeys(filter(None, row))), axis=1
            )

    return df_agrupado

# Leitura de todas as páginas
dfs = []
with pdfplumber.open("cardapio_junho.pdf") as pdf:
    for i in range(4):  # Páginas 0 e 1
        df_pag = processar_pagina(pdf.pages[i])
        if df_pag is not None:
            dfs.append(df_pag)

# Junta os resultados das duas páginas
df_final = pd.concat(dfs, ignore_index=True)





In [2]:
# Exibe resultado
df_final

,Dia,Saladas,Acompanhamento,Guarnição,Vegetariano,Prato Principal
0,"segunda-feira, 2 de junho de 2025",Acelga com tomate picado e Beterraba cozida,Arroz Branco e Feijão Preto,Aipim souté,Omelete de forno,Iscas suínas refogadas
1,"terça-feira, 3 de junho de 2025",Chicória com cenoura\nralada e Chuchu colorido...,Arroz Branco e Feijão Carioca,Espaguete ao molho sugo,Entreveiro de legumes,Espetinho de frango
2,"quarta-feira, 4 de junho de 2025",Alface com rabanete e Grão de bico com cenoura...,Arroz Branco/arroz\ncolorido e Feijão Preto,Quirerinha temperada,Escondidinho de cabotiá\ncom proteína de soja,Bife a tomatelli
3,"quinta-feira, 5 de junho de 2025",Almeirão com cenoura e Repolho cozido\nFruta,Arroz Branco e Feijão Preto,Batata doce\ncaramelada,Mini pizza de\nabobrinha,Pernil grelhado
4,"sexta-feira, 6 de junho de 2025",Alface com repolho e Abobrinha colorida,Arroz Branco e Feijão Preto,Macarrão com ervas\nfinas,Ovo ao molho,Frango assado
5,"segunda-feira, 9 de junho de 2025",Acelga com pepino e Abobrinha vinagrete,Arroz Branco e Feijão Preto,Legumes refogados,Omelete,Bife acebolado
6,"terça-feira, 10 de junho de 2025",Alface com repolho e Beterraba cozida\nFruta,Arroz Branco e Feijão Preto,Farofa colorida,Empada de legumes,Sobrecoxa assada
7,"quarta-feira, 11 de junho de 2025",Chicória com cebola e Vinagrete de lentilha,Arroz Branco e Feijão carioca,Polenta com molho sugo,Gratinado de chuchu,Peixe à milanesa (assado)
8,"quinta-feira, 12 de junho de 2025",Alface com cenoura e Chuchu cozido\nFruta,Arroz Branco e Feijão Preto,Rigatone ao molho\nbranco,Quibe vegano,Bisteca suína a tricolor
9,"sexta-feira, 13 de junho de 2025",Mix de folhas com e tomate e Mix grão,Arroz Branco e Feijão Carioca,Batata rústica,Strogonoff de carne de\nsoja,Strogonoff de frango


In [3]:
print(df_final)

                                       Dia  \
0        segunda-feira, 2 de junho de 2025   
1          terça-feira, 3 de junho de 2025   
2         quarta-feira, 4 de junho de 2025   
3         quinta-feira, 5 de junho de 2025   
4          sexta-feira, 6 de junho de 2025   
5        segunda-feira, 9 de junho de 2025   
6         terça-feira, 10 de junho de 2025   
7        quarta-feira, 11 de junho de 2025   
8        quinta-feira, 12 de junho de 2025   
9         sexta-feira, 13 de junho de 2025   
10  terça-feira, 17 de junho\nde 2025 None   
11       quinta-feira, 19 de junho de 2025   
12        sexta-feira, 20 de junho de 2025   
13                                           
14                                           
15      segunda-feira, 23 de junho de 2025   
16        terça-feira, 24 de junho de 2025   
17       quarta-feira, 25 de junho de 2025   
18       quinta-feira, 26 de junho de 2025   
19        sexta-feira, 27 de junho de 2025   

                                 

In [4]:
with pdfplumber.open("cardapio_junho.pdf") as pdf:
    for i, page in enumerate(pdf.pages, start=1):
        if i == 3:
            continue
        print(f"\n📄 Página {i}")
        tables = page.extract_tables()
        if not tables:
            print("⚠️ Nenhuma tabela encontrada nesta página.")
            continue
        for t_idx, table in enumerate(tables):
            print(f"\nTabela {t_idx} da página {i}")
            for row in table:
                print(row)



📄 Página 1



Tabela 0 da página 1
['', 'segunda-feira, 2 de junho de', '', '', 'terça-feira, 3 de junho de', '', '', 'quarta-feira, 4 de junho', '', '', 'quinta-feira, 5 de', '', '', 'sexta-feira, 6 de junho', '']
[None, '2025', None, None, '2025', None, None, 'de 2025', None, None, 'junho de 2025', None, None, 'de 2025', None]

Tabela 1 da página 1
['Saladas', 'Acelga com tomate picado', 'Chicória com cenoura\nralada', 'Alface com rabanete', 'Almeirão com cenoura', '', 'Alface com repolho', '']
[None, None, None, None, None, None, '', None]
[None, 'Beterraba cozida', 'Chuchu colorido\nFruta', 'Grão de bico com cenoura\ncozida', 'Repolho cozido\nFruta', 'Abobrinha colorida', None, None]
[None, None, None, None, None, None, 'Abobrinha colorida', None]
[None, None, None, None, None, None, '', None]
[None, None, None, None, None, '', '', '']
['Acompanhamento', 'Arroz Branco', 'Arroz Branco', 'Arroz Branco/arroz\ncolorido', 'Arroz Branco', 'Arroz Branco', None, None]
[None, 'Feijão Preto', 'Feijão Car